# Tarea para el Hogar 04

Esta Tarea para el Hogar 02 se entrega el final de la cuarta clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 05 que será el viernes 01-agosto

##  1. Cazatalentos

En la Clase 03 nos hemos enfrentado a  "La Maldicion del Ganandor",  elegir el modelo con el mejor puntaje simple no suele ser la mejor estrategia.
<br> Lea y ejecute el notebook  **src/CazaTalentos/CazaTalentos.ipynb**
<br> en caso de interesarle, participe del Desafío Ordenamiento  que vence el sábado 02 de agosto a las 19:00

## 2. Hiperparámetros del LightGBM

Los objetivos de esta tarea son:


*   Aumentar la rentabilidad de la campaña de marketing de retención proactiva de clientes.
*   Generar un mejor modelo optimizando sus hiperparámetros
*   Conceptual : investigar los mas relevantes hiperparámetros de LightGBM
*   Familiarizarse con la Bayesian Optimization, sus largos tiempos de corrida y opciones para reducirlos
*   Familiarizarse con el uso de máquinas virtuales de Google Colab
*   Ver un pipeline completo de optimización de hiperparámetros y puesta en producción

LightGBM cuenta con mas de 60 hiperparámetros, siendo posible utilizar 40 al mismo tiempo, aunque no razonable.
<br> La documentación oficial de los hiperparámetros de LightGBM es  https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters


Se lo alerta sobre que una Optimizacion Bayesiana lleva varias horas de corrida, y usted deberá correr VARIAS optimizaciones para descubrir cuales parámetros conviene optimizar.
<br> A pesar que la próxima clase es recien en viernes 01 de agosto, inicie la tarea con tiempo, aprenda a planificar estratégicamente sus corridas como un@ científ@  de datos.

Es necesario investigar cuales son los hiperparámetros de LightGBM que vale la pena optimizar en una Bayesian Optimization, ya que los realmente utiles son apenas un reducido subconjunto.
<br>Usted deberá investigar cuales son los hiperparámetros mas relevantes de LightGBM, su primer alternativa es preguntándole a su amigo con capacidades especiales ChatGPT o sus endogámicos familiares Claude, DeepSeek, Gemini, Grok, etc
<br> La segunda alternativa es la propia documentación de LightGBM  https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html


Adicionalmente podra buscar información como la que proveen esta diminuta muestra aleatoria de artículos ligeros:
*  https://medium.com/@sarahzouinina/a-deep-dive-into-lightgbm-how-to-choose-and-tune-parameters-7c584945842e
*  https://www.kaggle.com/code/somang1418/tuning-hyperparameters-under-10-minutes-lgbm
*  https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/


<br>  La muestra anterior se brinda a modo de ejemplo, usted deberá buscar muuuuchas  fuentes adicionales de información
<br> Tenga presente que LightGBM es el estado del arte en modelado predictivo para datasets estructurado, que son el 90% del trabajo del 95% de los Data Scientists en Argentina.

El desafío de esta tarea es:
* Qué hiperparparámetros conviene optimizar?  Las recomendaciones de los artículos ligeros es siempre sensata?  Sus autores realmente hicieron experimentos o son siemplemente escritores de entretenimiento carente de base científica?
* Elegidos los hiperparámetros, cual es el  <desde, hasta> que se debe utilizar en la Bayesian Optimization ?
* Realmente vale la pena optimizar 10 o 16 hiperparámetros al mismo tiempo ?  No resulta contraproducente una búsqueda en un espacio de tal alta dimensionalidad ?

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [148]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

ERROR: Error in parse(text = input): <text>:2:6: unexpected symbol
1: # primero establecer el Runtime de Python 3
2: from google.colab
        ^


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dm"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dm" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/itba2025-8d0a/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [20]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Aug 01 14:13:27 2025"

In [21]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2284981,122.1,3892774,207.9,3892774,207.9
Vcells,16965059,129.5,93088812,710.3,104516596,797.4


### 2.2.2 Carga de Librerias

In [22]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [25]:
PARAM <- list()
PARAM$experimento <- 491989
PARAM$semilla_primigenia <- 100019


In [26]:
PARAM$kaggle$competencia <- "data-mining-analista-sr-2025-a"
PARAM$kaggle$cortes <- seq(10000, 12000, by= 500)

In [27]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.5

In [28]:

# =====================================
# Parámetros LightGBM (fijos)
# =====================================
PARAM$hyperparametertuning$xval_folds <- 5

PARAM$lgbm$param_fijos <- list(
  boosting = "gbdt",
  objective = "binary",
  metric = "auc",
  first_metric_only = TRUE,
  boost_from_average = TRUE,
  feature_pre_filter = FALSE,
  force_row_wise = TRUE,
  verbosity = -100,
  seed = PARAM$semilla_primigenia,
  num_iterations = 1200,           # Más iteraciones para learning_rate bajo
  early_stopping_round = 100       # Ajustamos el early stopping
)

# =====================================
# Rango de hiperparámetros para Bayesiana
# =====================================
PARAM$lgbm$param_BO <- list(
  num_leaves               = c(10L, 120L),
  min_data_in_leaf         = c(1L, 80L),
  learning_rate            = c(0.01, 0.15),
  feature_fraction         = c(0.8, 1.0),
  bagging_fraction         = c(0.8, 1.0),
  max_depth                = c(-1L, 8L),              # probamos sin límite y poco profundo
  min_sum_hessian_in_leaf  = c(1e-3, 0.1),
  lambda_l1                = c(0, 5),
  lambda_l2                = c(0, 5),
  min_gain_to_split        = c(0, 0.1),
  min_data_in_bin          = c(1L, 10L),
  max_bin                  = c(63L, 255L),
  scale_pos_weight         = c(0.5, 5)
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [29]:
PARAM$hyperparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 10L, upper = 80L),
  makeIntegerParam("min_data_in_leaf", lower = 1L, upper = 50L),
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.15),
  makeNumericParam("feature_fraction", lower = 0.8, upper = 1.0),
  makeNumericParam("bagging_fraction", lower = 0.8, upper = 1.0),
  makeIntegerParam("max_depth", lower = -1L, upper = 8L),   # Ahora entero, incluye -1
  makeNumericParam("min_sum_hessian_in_leaf", lower = 1e-3, upper = 0.1),
  makeNumericParam("lambda_l1", lower = 0, upper = 5),
  makeNumericParam("lambda_l2", lower = 0, upper = 5),
  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.1),
  makeIntegerParam("min_data_in_bin", lower = 1L, upper = 10L),
  makeIntegerParam("max_bin", lower = 63L, upper = 255L),
  makeNumericParam("scale_pos_weight", lower = 0.5, upper = 5)
)

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [30]:
PARAM$hyperparametertuning$iteraciones <- 400 # iteraciones bayesianas

### 2.2.4  Preprocesamiento

In [31]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [32]:
# lectura del dataset

dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [33]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [34]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [35]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [36]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [37]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 83617

[1] 154

2.2.5 Configuracion Bayesian Optimization

In [38]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [39]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set = PARAM$hyperparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [40]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Fri Aug 01 14:14:58 2025 AUC 0.929170825313149

Fri Aug 01 14:15:12 2025 AUC 0.92876258117334

Fri Aug 01 14:15:25 2025 AUC 0.928178251951555

Fri Aug 01 14:15:50 2025 AUC 0.929305434953152

Fri Aug 01 14:16:17 2025 AUC 0.927996902751242

Fri Aug 01 14:16:33 2025 AUC 0.927266792457021

Fri Aug 01 14:16:43 2025 AUC 0.927654262727688

Fri Aug 01 14:17:03 2025 AUC 0.928837651827217

Fri Aug 01 14:17:15 2025 AUC 0.928389657962815

Fri Aug 01 14:18:12 2025 AUC 0.929336404698732

Fri Aug 01 14:18:25 2025 AUC 0.925504556258362

Fri Aug 01 14:18:37 2025 AUC 0.928661406770035

Fri Aug 01 14:18:54 2025 AUC 0.92929829050553

Fri Aug 01 14:19:05 2025 AUC 0.926377078963328

Fri Aug 01 14:19:14 2025 AUC 0.925304709276128

Fri Aug 01 14:19:41 2025 AUC 0.929302161616692

Fri Aug 01 14:19:57 2025 AUC 0.924129332228541

Fri Aug 01 14:20:20 2025 AUC 0.928728874376849

Fri Aug 01 14:20:46 2025 AUC 0.9290125136999

Fri Aug 01 14:21:00 2025 AUC 0.929417510743

In [41]:
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_leaves"              "min_data_in_leaf"       
 [3] "learning_rate"           "feature_fraction"       
 [5] "bagging_fraction"        "max_depth"              
 [7] "min_sum_hessian_in_leaf" "lambda_l1"              
 [9] "lambda_l2"               "min_gain_to_split"      
[11] "min_data_in_bin"         "max_bin"                
[13] "scale_pos_weight"        "y"                      
[15] "dob"                     "eol"                    
[17] "error.message"           "exec.time"              
[19] "ei"                      "error.model"            
[21] "train.time"              "prop.type"              
[23] "propose.time"            "se"                     
[25] "mean"

In [42]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [43]:
write_yaml( PARAM, file="PARAM.yml")

In [44]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_leaves min_data_in_leaf learning_rate feature_fraction bagging_fraction
        <int>            <int>         <num>            <num>            <num>
1:         10                7    0.02117858        0.9495472        0.8769273
   max_depth min_sum_hessian_in_leaf lambda_l1 lambda_l2 min_gain_to_split
       <int>                   <num>     <num>     <num>             <num>
1:         6              0.03420337 0.5697657  2.152797        0.06580211
   min_data_in_bin max_bin scale_pos_weight
             <int>   <int>            <num>
1:               5     202        0.5218686
[1] 0.9318122


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [45]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [46]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [47]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [48]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [49]:

param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 100019

$num_iterations
[1] 1200

$early_stopping_round
[1] 100

$num_leaves
[1] 10

$min_data_in_leaf
[1] 7

$learning_rate
[1] 0.02117858

$feature_fraction
[1] 0.9495472

$bagging_fraction
[1] 0.8769273

$max_depth
[1] 6

$min_sum_hessian_in_leaf
[1] 0.03420337

$lambda_l1
[1] 0.5697657

$lambda_l2
[1] 2.152797

$min_gain_to_split
[1] 0.06580211

$min_data_in_bin
[1] 5

$max_bin
[1] 202

$scale_pos_weight
[1] 0.5218686

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [50]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling

In [51]:
  # entreno LightGBM
dvalid <- dtrain
modelo_final <- lgb.train(
  data = dtrain,
  param = param_normalizado,
  valids = list(valid = dvalid)
)


In [52]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(modelo_final))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [53]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo_final, "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [54]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

#### Tabla Prediccion

In [55]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [56]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to Data Mining, Analista Sr 2025A 
Successfully submitted to Data Mining, Analista Sr 2025A 
Successfully submitted to Data Mining, Analista Sr 2025A 
Successfully submitted to Data Mining, Analista Sr 2025A 
Successfully submitted to Data Mining, Analista Sr 2025A 


In [57]:
write_yaml( PARAM, file="PARAM.yml")

In [58]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Aug 01 20:33:47 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar04**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla